# Import libraries

In [1]:
import pandas as pd
import numpy as np

# Load data

In [2]:
df = pd.read_csv('in-vehicle-coupon-recommendation.csv')
df.head()

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
0,No Urgent Place,Alone,Sunny,55,2PM,Restaurant(<20),1d,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,0,0,0,1,1
1,No Urgent Place,Friend(s),Sunny,80,10AM,Coffee House,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,0,0,0,1,0
2,No Urgent Place,Friend(s),Sunny,80,10AM,Carry out & Take away,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,1
3,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,0
4,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,1d,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,0


# Build a K-Nearest Neighbors model

In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Preprocessing

In [4]:
df = df.drop(['car'], axis=1)

In [5]:
df = df.dropna()

In [6]:
df = pd.get_dummies(df)
df.head()

,temperature,has_children,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y,destination_Home,destination_No Urgent Place,...,RestaurantLessThan20_1~3,RestaurantLessThan20_4~8,RestaurantLessThan20_gt8,RestaurantLessThan20_less1,RestaurantLessThan20_never,Restaurant20To50_1~3,Restaurant20To50_4~8,Restaurant20To50_gt8,Restaurant20To50_less1,Restaurant20To50_never
22,55,0,1,0,0,0,1,1,0,1,...,0,1,0,0,0,0,0,0,1,0
23,80,0,1,0,0,0,1,0,0,1,...,0,1,0,0,0,0,0,0,1,0
24,80,0,1,0,0,0,1,1,0,1,...,0,1,0,0,0,0,0,0,1,0
25,80,0,1,1,0,0,1,0,0,1,...,0,1,0,0,0,0,0,0,1,0
26,80,0,1,0,0,0,1,0,0,1,...,0,1,0,0,0,0,0,0,1,0


### PCA

In [7]:
from sklearn.decomposition import PCA

In [8]:
pca = PCA(n_components=2)

In [9]:
pca.fit(df)

PCA(n_components=2)

In [10]:
pca.explained_variance_ratio_

array([0.9663956 , 0.00241736])

In [11]:
X = df.drop(['Y'], axis=1)

In [12]:
y = df['Y'] 

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [14]:
# Function to test each column
def test_column(col):
    model = KNeighborsClassifier(n_neighbors = 10, p = 2)
    model.fit(X_train[col].to_frame(), y_train)
    y_pred = model.predict(X_test[col].to_frame())
    return accuracy_score(y_test, y_pred)

In [15]:
def test_columns(cols):
    model = KNeighborsClassifier(n_neighbors = 10, p = 2)
    model.fit(X_train[cols], y_train)
    y_pred = model.predict(X_test[cols])
    return accuracy_score(y_test, y_pred)

In [16]:
def test_all():
    model = KNeighborsClassifier(n_neighbors = 10, p = 2)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)

In [17]:
# Test each column
# for column in X_train.columns:
#     print(column, test_column(column))

temperature 0.5542218543046358
has_children 0.581953642384106
toCoupon_GEQ5min 0.581953642384106
toCoupon_GEQ15min 0.581953642384106
toCoupon_GEQ25min 0.5885761589403974
direction_same 0.581953642384106
direction_opp 0.5422185430463576
destination_Home 0.581953642384106
destination_No Urgent Place 0.581953642384106
destination_Work 0.581953642384106
passanger_Alone 0.581953642384106
passanger_Friend(s) 0.581953642384106
passanger_Kid(s) 0.5753311258278145
passanger_Partner 0.581953642384106
weather_Rainy 0.5935430463576159
weather_Snowy 0.581953642384106
weather_Sunny 0.5927152317880795
time_10AM 0.581953642384106
time_10PM 0.581953642384106
time_2PM 0.581953642384106
time_6PM 0.581953642384106
time_7AM 0.581953642384106
coupon_Bar 0.6026490066225165
coupon_Carry out & Take away 0.581953642384106
coupon_Coffee House 0.581953642384106
coupon_Restaurant(20-50) 0.5968543046357616
coupon_Restaurant(<20) 0.581953642384106
expiration_1d 0.581953642384106
expiration_2h 0.4503311258278146
gend

In [18]:
# Test columns in pairs
# for col1 in X_train.columns:
#     for col2 in X_train.columns:
#         if col1 < col2:
#             print(col1, col2, test_columns([col1, col2]))

In [19]:
# Accuracy of all columns
test_all()

0.6626655629139073